In [16]:
import escher
from escher import Builder
import pandas
from time import time, sleep

import cobra
from cobra.medium import minimal_medium
from cobra.io import load_model
from cobra import Model, Reaction, Metabolite
from cobra.manipulation import delete_model_genes, knock_out_model_genes, remove_genes

model = load_model("iJN1463")

# The genes to be deleted one at a time
# genes_to_delete = ["thiO", "ltaE", "serA", "aceA", "gcvR", "pha_cluster"]
# gene_ids = ["PP_0612", "PP_0321", "PP_5155", "PP_4116", "PP_0997", "PP_5002", "PP_5003", "PP_5004", "PP_5005", "PP_5006", "PP_5007", "PP_5008"]
genes_to_delete = ["thiO", "ltaE", "serA", "aceA", "phaA","phaB","phaC2"]
gene_ids = ["PP_0612", "PP_0321", "PP_5155", "PP_4116", "PP_5003", "PP_5004", "PP_5005"]

# Single deletion of each gene in my gene list
gene_list = gene_ids

remove_genes(model, gene_list)

In [21]:
medium = model.medium
# medium["EX_nh4_e"] = 4.5
# medium["EX_for_e"] = 16.6
# medium["EX_ac_e"] = 100.1
medium["EX_glc__D_e"] = 100.0
# medium["EX_co2_e"] = 10.0
medium["EX_gly_e"] = 10.0
medium["EX_ser__L_e"] = 10.0
model.medium = medium

model.medium

# model.optimize()


# Module 1: C1 Attachment to THF
# adding ftfL, fch, and mtdA genes from M. extorquens AM1 to model

ftfL_reaction = Reaction("R00943")
ftfL_reaction.name = "Formate-tetrahydrofolate ligase"
ftfL_reaction.subsystem = "one-carbon metabolism"
ftfL_reaction.lower_bound = 0.  # to make reversible

ftfL_reaction.add_metabolites({
    model.metabolites.get_by_id("atp_c"): -1.0,
    model.metabolites.get_by_id("for_c"): -1.0,
    model.metabolites.get_by_id("thf_c"): -1.0,
    model.metabolites.get_by_id("10fthf_c"): 1.0,
    model.metabolites.get_by_id("adp_c"): 1.0,
    model.metabolites.get_by_id("pi_c"): 1.0
})

# ftfL_reaction.reaction

# model.reactions.get_by_id("R00943")

fch_reaction = Reaction("R01655")
fch_reaction.name = "Methenyltetrahydrofolate cyclohydrolase"
fch_reaction.subsystem = "one-carbon metabolism"
fch_reaction.lower_bound = 0.  # to make reversible

fch_reaction.add_metabolites({
    model.metabolites.get_by_id("10fthf_c"): -1.0,
    model.metabolites.get_by_id("h_c"): -1.0,
    model.metabolites.get_by_id("h2o_c"): 1.0,
    model.metabolites.get_by_id("methf_c"): 1.0
})

# fch_reaction.reaction

mtdA_reaction = Reaction("R01220")
mtdA_reaction.name = "5,10-methylenetetrahydrofolate:NADP+ oxidoreductase"
mtdA_reaction.subsystem = "one-carbon metabolism"
mtdA_reaction.lower_bound = 0.  # to make reversible

mtdA_reaction.add_metabolites({
    model.metabolites.get_by_id("methf_c"): -1.0,
    model.metabolites.get_by_id("nadph_c"): -1.0,
    model.metabolites.get_by_id("mlthf_c"): 1.0,
    model.metabolites.get_by_id("nadp_c"): 1.0
})

model.add_reactions([ftfL_reaction, fch_reaction, mtdA_reaction])

# Module 2: Glycine Generation
# switching substrate and product for glycine cleavage system

glycine_cleavage_reaction = model.reactions.get_by_id("GLYCL")
glycine_cleavage_reaction.lower_bound = -1000.
glycine_cleavage_reaction.upper_bound = 0.

model.reactions.get_by_id("GLYCL")


# Module 3: Pyruvate Synthesis
glyA_reaction = model.reactions.get_by_id("GHMT2r")
glyA_reaction.lower_bound = -1000.
glyA_reaction.upper_bound = 0.

# tdc_reaction = model.reactions.get_by_id("SERD_L")


model.optimize()

Ignoring reaction 'R00943' since it already exists.
Ignoring reaction 'R01655' since it already exists.
Ignoring reaction 'R01220' since it already exists.


,fluxes,reduced_costs
3HAD160,0.000000e+00,0.000000e+00
13DAMPPabcpp,0.000000e+00,7.185359e-16
13DAMPPtex,0.000000e+00,0.000000e+00
15DAPabcpp,0.000000e+00,1.238257e-15
1P2CBXLCYCL,0.000000e+00,-0.000000e+00
...,...,...
4MCAT23DOX,0.000000e+00,-1.110223e-16
4OD,0.000000e+00,0.000000e+00
R00943,-6.071820e-19,-3.330669e-15
R01655,0.000000e+00,0.000000e+00


In [18]:
# # max_growth = model.slim_optimize()
# # minimal_medium(model, min_objective_value=0.5, exports=False, minimize_components=False, open_exchanges=True)
# model.medium = minimal_medium(model, min_objective_value=0.5, exports=False, minimize_components=False, open_exchanges=True)

# model.optimize()

In [19]:
# model.medium

In [171]:
# model.reactions.get_by_id("GLYCL")
# model.reactions.get_by_id("GHMT2r")
# model.reactions.get_by_id("SERD_L")

Reaction identifier,SERD_L
Name,L-serine deaminase
Memory address,0x7fd036685e50
Stoichiometry,ser__L_c --> nh4_c + pyr_c L-Serine --> Ammonium + Pyruvate
GPR,PP_3144 or PP_0297 or PP_0987
Lower bound,0.0
Upper bound,999999.0


In [20]:
builder = Builder()

builder.model = model

# Run FBA with the model and add the flux data to the map
solution = builder.model.optimize()
builder.reaction_data = solution.fluxes

# Add some data for metabolites
builder.metabolite_data = solution.shadow_prices

# Simplify the map by hiding some labels
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False

builder.reaction_scale = [
    { 'type': 'min', 'color': '#000000', 'size': 12 },
    { 'type': 'median', 'color': '#ffffff', 'size': 20 },
    { 'type': 'max', 'color': '#ff0000', 'size': 25 }
]

builder.reaction_scale_preset = 'GaBuRd'

# Make all the arrows three times as thick
builder.reaction_scale = [
    {k: v * 3 if k == 'size' else v for k, v in x.items()}
    for x in builder.reaction_scale
]

builder

Builder(hide_all_labels=False, hide_secondary_metabolites=True, metabolite_data={'10fthf_c': -0.0, '12dgr120_c…

In [14]:
model.metabolites.get_by_id("gly_c")

Metabolite identifier,gly_c
Name,Glycine
Memory address,0x7f8a4c57c400
Formula,C2H5NO2
Compartment,c
In 25 reaction(s),"DALAGLYD, GLYPHEHYc, GLYt2pp, BIOMASS_KT2440_Core2, PPRGL, GLYGLYCNc, SARCOX2, GLYCL, AMPTASECG, ALDD31, AMPTASEPG, AMPTASEGM, GTHS, GHMT2r, GLYTRS, AMPTASEGGLU, GLYt4pp, AMPTASEGGLN, SARCOX,..."
